In [4]:
import open3d as o3d
import numpy as np

bounding_box = o3d.geometry.OrientedBoundingBox(np.zeros(3),np.identity(3),np.ones(3)/.315)

pcd = o3d.io.read_point_cloud("/home/sashawald/Documents/shaving_code-main/clouds/2022-06-03.ply")
down_pcd = pcd.voxel_down_sample(0.035)
crop_pcd = down_pcd.crop(bounding_box)

vis = o3d.visualization.VisualizerWithEditing()
vis.create_window()
vis.add_geometry(crop_pcd)
vis.run()  
vis.destroy_window()
pdata = vis.get_picked_points()
dpcd=crop_pcd

[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #802 (-0.36, -0.1, -1.5) to add in queue.
[Open3D INFO] Picked point #1736 (-0.0030, -0.1, -1.5) to add in queue.


In [5]:
#Ray Casting
from LineMesh import *
#Visualize Cylinder Mesh + Point Cloud
points = [crop_pcd.points[(pdata[0])],
          crop_pcd.points[(pdata[1])]]
lines = [[0,1]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)

#Produce corresponding cylinder with LineMesh
line_mesh=LineMesh(points,radius=0.01)
line_mesh_geoms=line_mesh.cylinder_segments

#Ray Casting
#Without some downsampling, this process will take 10-15 seconds
cyl = o3d.t.geometry.TriangleMesh.from_legacy(line_mesh_geoms[0])
scene = o3d.t.geometry.RaycastingScene()
cyl_id = scene.add_triangles(cyl)
print(cyl_id)

pclrays = np.ndarray.tolist(np.asarray(crop_pcd.points))
for ray in pclrays:
    ray.extend([0,0,1])

rays = o3d.core.Tensor(pclrays,
                       dtype=o3d.core.Dtype.Float32)
ans = (scene.cast_rays(rays))['geometry_ids']
count=0
found=[]
for val in ans:
    if val == 0:
        found.append(count)
    count=count+1

crop_pcd.paint_uniform_color([0,1,0])
foundpcd = crop_pcd.select_by_index(found)
foundpcd.paint_uniform_color([1,0,1])
o3d.visualization.draw_geometries([crop_pcd,foundpcd])


0


In [6]:
import networkx as nx
G = nx.Graph()

pcd_tree = o3d.geometry.KDTreeFlann(dpcd)
count=0
for vertex in dpcd.points:
    [k, idx, _] = pcd_tree.search_knn_vector_3d(vertex, 5)
    for v_curr in idx:
        current_vertex=dpcd.points[v_curr]
        if not (count==v_curr):
            dist = np.sqrt(np.sum((vertex-current_vertex)**2, axis=0))
            G.add_edge(count,v_curr,weight=dist)
    count=count+1

path=(nx.dijkstra_path(G,pdata[0],pdata[1]))
foundpcd = dpcd.select_by_index(path)
foundpcd.paint_uniform_color([1, 0, 0])
dpcd.paint_uniform_color([0,1,0])
o3d.visualization.draw_geometries([dpcd,foundpcd])

In [7]:
import networkx as nx
G = nx.Graph()

pcd_tree = o3d.geometry.KDTreeFlann(dpcd)
count=0
for vertex in dpcd.points:
    [k, idx, _] = pcd_tree.search_knn_vector_3d(vertex, 5)
    for v_curr in idx:
        current_vertex=dpcd.points[v_curr]
        if not (count==v_curr):
            dist = np.sqrt(np.sum((vertex-current_vertex)**2, axis=0))
            G.add_edge(count,v_curr,weight=dist)
    count=count+1

search=(nx.bfs_tree(G,pdata[0],sort_neighbors=None))
path=nx.shortest_path(search,source=pdata[0],target=pdata[1])
foundpcd = dpcd.select_by_index(path)
foundpcd.paint_uniform_color([1, 0, 0])
dpcd.paint_uniform_color([0,1,0])
o3d.visualization.draw_geometries([dpcd,foundpcd]) 